In [46]:
import re
import csv
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score
#from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

In [47]:
#start processing data twitter
def processTweet(tweet):
    # process the tweets
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
#     print(tweet)
#     print(' ')
    return tweet
#end

In [48]:
#initialize stopWords
stopWords = []

In [49]:
#start replaceTwoOrMore
def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)
#end

In [50]:
#start getStopWordList
def getStopWordList(stopWordListFileName):
    #read the stopwords file and build a list
    stopWords = []
    stopWords.append('AT_USER')
    stopWords.append('URL')

    fp = open(stopWordListFileName, 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    return stopWords
#end

In [51]:
#start getfeatureVector
def getFeatureVector(tweet):
    featureVector = []
    # tweet=stemmer.stem(tweet)
    #split tweet into words
    words = tweet.split()
    for w in words:
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(w in stopWords or val is None):
            continue
        else:
           featureVector.append(w.lower())
#     print(featureVector)
#     print(' ')
    return featureVector
#end

In [52]:
#start replace singkatan/kata alay
def translator(tweet):
    j = 0
    for kata in tweet:
        with open('singkatan.txt', 'r') as csvfile:
            # Reading file as CSV with delimiter as "=", so that abbreviation are stored in row[0] and phrases in row[1]
            dataFromFile = csv.reader(csvfile, delimiter="=")
            for row in dataFromFile:
                # Check if selected word matches short forms[LHS] in text file.
                if kata == row[0]:
                    # If match found replace it with its appropriate phrase in text file.
                    tweet[j] = row[1]
            csvfile.close()
        j = j + 1
#     print(' '.join(tweet))
#     print('')
    return tweet    
#end process

In [53]:
#Read the tweets one by one and process it
df = pd.read_csv('pertama.csv', delimiter=',', index_col = False )
line = df['text']
st = open('konjungsi.csv', 'r')
stopWords = getStopWordList('konjungsi.csv')
result_list = []

In [54]:
for x in range(0, len(line)):
    processedTweet = processTweet(line[x])
#     print(processedTweet)
    featureVector = getFeatureVector(processedTweet)
#     print(featureVector)
    replaceAbb = translator(featureVector)
#     print(replaceAbb)
    result_list.append(replaceAbb)   
#     result_list.append(temp)
#     line = fp.readline()
#end loop
print(result_list)

[['selamat', 'bermedsos', 'temans', 'promosikan', 'kebaikan2', 'prabowo', 'promosikan', 'kebaikan2', 'mas', 'sandi', 'promosikan', 'partai2', 'koalisi', 'adil', 'pan', 'pks', 'demokrat', 'gerindra', 'promosikan', 'caleg2', 'pan', 'pks', 'demokrat', 'gerindra'], ['habib', 'rizieq', 'shihab', 'gelar', 'aksi', 'bela', 'tauhid', 'indonesia'], ['permintaan', 'maaf', 'saran', 'gus', 'solah', 'proses', 'hukum', 'ngomong', 'idiot', 'tersangka', 'tebang', 'pilih', 'pembakaran', 'bendera', 'tauhid', 'melukai', 'hati', 'umat', 'cepat', 'ratna', 'sarumpaet', 'adillah'], ['ketemu', 'mari', 'kuatkan', 'aksibelakalimattauhid', 'banserbertaubatlah', 'bapakhoaxnasional', 'karena', 'yuks', 'kenal'], ['gerakan', 'aksibelakalimattauhid', 'aksigarutbelatauhid', 'diikuti', 'saudara', 'aksisolorayabelatauhid', 'bergema', 'solo', 'siapkan', 'untuk', 'indonesiaadilmakmur'], ['trigger', 'menyatukan', 'umat', 'islam', 'disitulah', 'allah', 'jalan', 'bendera', 'tauhid', 'bakar', 'serentak', 'bendera', 'berkibar',

In [55]:
tempStr = []
for baris in range(0, len(result_list)):
    tempo=""
    for kolom in range(0,len(result_list[baris])):
        tempo = tempo+" "+result_list[baris][kolom]
    tempStr.append(tempo)
# print(type(tempStr))

In [56]:
tempData = df['text']
for x in range(0,len(tempStr)):
    df['text'][x] = tempStr[x]
#     tempData.replace(x,tempStr[x])

print(df['text'])


c:\users\m. anshori\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0       selamat bermedsos temans promosikan kebaikan2...
1       habib rizieq shihab gelar aksi bela tauhid in...
2       permintaan maaf saran gus solah proses hukum ...
3       ketemu mari kuatkan aksibelakalimattauhid ban...
4       gerakan aksibelakalimattauhid aksigarutbelata...
5       trigger menyatukan umat islam disitulah allah...
6       aksigarutbelatauhid belakalimattauhid gelomba...
7       butuh figure sosok berjiwa nasionalis sudah s...
8       muslim bangkit aksigarutbertauhid mari gemaka...
9       maaf naikin tagar tidak intropeksi namanya sa...
10      orang bertanggung maaf terbuka ngajarin mendo...
11      komen bebas komen terbaik hadiah hadiah ban s...
12      tum savicali qitmr ajarin anak buah kamu jang...
13      selamat milad babe barokallahu fii umrik mabr...
14      catat tolong yaqut jangan kebanyakan ngeles k...
15      sama sya allah tanggapan aagym pembakaran ben...
16      cuplikan video cak nun angkat bicara pembakar...
17      versi ketua gp bendera 

In [57]:
df.head()

,username,user_handle,date,retweets,favorites,text,geological_location,mentions,hashtags,tweet_id,permalink,tokoh,sentiment
0,Michel Adam,MichelaniaAdam,10/24/2018 8:35,132,309,selamat bermedsos temans promosikan kebaikan2...,NaN,NaN,# #,1.05E+18,https://twitter.com/MichelaniaAdam/status/1054...,prabowo,positive
1,PurwaRaja,RajaPurwaBU,10/24/2018 6:13,101,218,habib rizieq shihab gelar aksi bela tauhid in...,NaN,NaN,#,1.05E+18,https://twitter.com/RajaPurwaBU/status/1054873...,prabowo,positive
2,CAK KHUM,CakKhum,10/23/2018 20:37,982,1714,permintaan maaf saran gus solah proses hukum ...,NaN,NaN,#,1.05E+18,https://twitter.com/CakKhum/status/10547285250...,jokowi,negative
3,#RelawanProSandi Official,RelawanProSandi,10/23/2018 20:34,1124,993,ketemu mari kuatkan aksibelakalimattauhid ban...,NaN,NaN,# # # # # # #,1.05E+18,https://twitter.com/RelawanProSandi/status/105...,jowowi,negative
4,#RanupsPrabowoSandi Akun Official,RanupsProSandi,10/23/2018 19:47,201,374,gerakan aksibelakalimattauhid aksigarutbelata...,NaN,NaN,# # # # # #,1.05E+18,https://twitter.com/RanupsProSandi/status/1054...,prabowo,positive


In [38]:
df.to_csv("out.csv",delimiter=',', index_col = False, encoding = "ISO-8859-1")

In [58]:
df['Sentiment'] = df['Sentiment'].replace({'Positive': 1, 'Negative': -1,'Neutral': 0})

In [59]:
df.head()

,username,user_handle,date,retweets,favorites,text,geological_location,mentions,hashtags,tweet_id,permalink,tokoh,sentiment
0,Michel Adam,MichelaniaAdam,10/24/2018 8:35,132,309,selamat bermedsos temans promosikan kebaikan2...,NaN,NaN,# #,1.05E+18,https://twitter.com/MichelaniaAdam/status/1054...,prabowo,1
1,PurwaRaja,RajaPurwaBU,10/24/2018 6:13,101,218,habib rizieq shihab gelar aksi bela tauhid in...,NaN,NaN,#,1.05E+18,https://twitter.com/RajaPurwaBU/status/1054873...,prabowo,1
2,CAK KHUM,CakKhum,10/23/2018 20:37,982,1714,permintaan maaf saran gus solah proses hukum ...,NaN,NaN,#,1.05E+18,https://twitter.com/CakKhum/status/10547285250...,jokowi,0
3,#RelawanProSandi Official,RelawanProSandi,10/23/2018 20:34,1124,993,ketemu mari kuatkan aksibelakalimattauhid ban...,NaN,NaN,# # # # # # #,1.05E+18,https://twitter.com/RelawanProSandi/status/105...,jowowi,0
4,#RanupsPrabowoSandi Akun Official,RanupsProSandi,10/23/2018 19:47,201,374,gerakan aksibelakalimattauhid aksigarutbelata...,NaN,NaN,# # # # # #,1.05E+18,https://twitter.com/RanupsProSandi/status/1054...,prabowo,1


In [60]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], 
                                                    df['sentiment'],
                                                    random_state=0)

In [61]:
X_test.index

Int64Index([ 18, 170, 107,  98, 177, 182,   5, 146,  12, 152,  61, 125, 180,
            154,  80,   7,  33, 130,  37,  74, 183, 145,  45, 159,  60, 123,
            179, 185, 122,  44,  16,  55, 150, 111,  22, 189, 129,   4,  83,
            106, 134,  66,  26, 113, 168,  63,   8,  75, 118, 143],
           dtype='int64')

In [62]:
print(len(y_train), len(y_test))

150 50


In [78]:
print('X_train first entry:\n\n', X_train.iloc[3])
print('\n\nX_train shape: ', X_train.shape)

X_train first entry:

  versi ketua gp bendera berceceran diselematkan bendera tsb jatuh tidak semestinya bendera baik dibakar bakar dilipat simpan eh alasan bendera hti


X_train shape:  (150,)


In [66]:
vect = TfidfVectorizer(min_df=5, stop_words='english', use_idf=True, ngram_range=(1,2)).fit(X_train)
len(vect.get_feature_names())

226

In [67]:
X_train_vectorized = vect.transform(X_train)

In [68]:
mnb = MultinomialNB()

mnb.fit(X_train_vectorized, y_train)

predictions = mnb.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))
print('Confusion Metrix: ', confusion_matrix(y_test,predictions))
print('ACC: ', accuracy_score(y_test, predictions))

AUC:  0.9615384615384616
Confusion Metrix:  [[24  0]
 [ 2 24]]
ACC:  0.96


In [80]:
print(mnb.predict(vect.transform([' permintaan maaf saran gus solah proses hukum ngomong idiot tersangka tebang pilih pembakaran bendera tauhid melukai hati umat cepat ratna sarumpaet adillah'])))

[0]
